In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Remove unnecessary features

### Name

In [3]:
# Names are probably not useful for predicting the survival of a person so we'll remove that column
del data['Name']
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


### Ticket

In [4]:
# Ticket feature seems somewhat inconsistent and is probably not too important so we can go ahead and remove it as well
del data['Ticket']
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S


## Checking for missing values

In [5]:
# Age, Cabin, and Embarked have missing values
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Embarked

In [6]:
# Embarked only has two missing values
embarked_null = data['Embarked'].isnull()
data[embarked_null]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
61,62,1,1,female,38.0,0,0,80.0,B28,NaN
829,830,1,1,female,62.0,0,0,80.0,B28,NaN


In [7]:
# Since it's categorical, we can use the most frequent embarkation place (S) to fill the values
# (alternatively, we could have decided to just remove these two rows)
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
# S (Southampton) is the most frequent
data['Embarked'] = data['Embarked'].fillna('S')
data[embarked_null]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
61,62,1,1,female,38.0,0,0,80.0,B28,S
829,830,1,1,female,62.0,0,0,80.0,B28,S


### Cabin

In [9]:
# There are too many missing values (687 out of 891), it's better to just ignore this feature and remove the column
del data['Cabin']
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


### Age

In [10]:
# Since age is a continuous feature, we'll use linear regression to try to predict the age of missing ages using the non-missing ones
# Taken from: https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e
from sklearn.linear_model import LinearRegression

train_data = data[["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]]

train_data["Sex"] = [1 if x == "male" else 0 for x in data["Sex"]]

test_data = train_data[train_data["Age"].isnull()]
train_data.dropna(inplace=True)

y_train = train_data["Age"]
X_train = train_data.drop("Age", axis=1)
X_test = test_data.drop("Age", axis=1)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

<ipython-input-10-a37f3a1f9501>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Sex"] = [1 if x == "male" else 0 for x in data["Sex"]]
<ipython-input-10-a37f3a1f9501>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.dropna(inplace=True)


In [11]:
# some of the predicted ages are negative values
neg_pred = y_pred <= 0
y_pred[neg_pred]

array([-5.49189866, -5.20126796, -5.49189866, -5.49189866, -5.20126796,
       -5.49189866, -5.20126796])

In [12]:
# we could replace the negative values zero values
y_pred[neg_pred] = 0
y_pred[neg_pred]

array([0., 0., 0., 0., 0., 0., 0.])

In [13]:
null_age = data['Age'].isnull()
data[null_age].head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,male,NaN,0,0,8.4583,Q
17,18,1,2,male,NaN,0,0,13.0000,S
19,20,1,3,female,NaN,0,0,7.2250,C
26,27,0,3,male,NaN,0,0,7.2250,C
28,29,1,3,female,NaN,0,0,7.8792,Q


In [14]:
data.loc[null_age, 'Age'] = y_pred
data[null_age].head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,male,29.070801,0,0,8.4583,Q
17,18,1,2,male,30.108333,0,0,13.0000,S
19,20,1,3,female,22.446851,0,0,7.2250,C
26,27,0,3,male,29.089273,0,0,7.2250,C
28,29,1,3,female,22.437052,0,0,7.8792,Q


In [15]:
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

## Save new preprocessed dataset

In [16]:
data.to_csv('data/preprocessed_train.csv')